In [1]:
import torch

import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

import torch.nn as nn

import matplotlib.pyplot as plt

import random

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [3]:
class DNN(nn.Module):
    def __init__(self, in_shape:tuple, out_shape:int, activation:nn.Module, loss:nn.Module, optimizer:torch.optim.Optimizer):
        super().__init__()
        self.flatten = nn.Flatten()
        self.network = nn.Sequential(
            nn.Linear(in_shape[0] * in_shape[1], 512),
            activation(),
            nn.Linear(512, 512),
            activation(),
            nn.Linear(512, out_shape)
        )
        self.loss = loss()
        self.optimizer = optimizer(self.parameters(), lr=1e-3)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.network(x)
        return logits

In [4]:
model = DNN((28,28),10, nn.ReLU, nn.CrossEntropyLoss, torch.optim.SGD).to(device)

print(model)

DNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (network): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
  (loss): CrossEntropyLoss()
)


In [5]:
def train(dataloader, model):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = model.loss(pred, y)

        # 역전파
        model.optimizer.zero_grad()
        loss.backward()
        model.optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [6]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += model.loss(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

100%|██████████| 26421880/26421880 [00:01<00:00, 18229260.24it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 388555.29it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6216280.38it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 14375683.75it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [8]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299103  [    0/60000]
loss: 2.292617  [ 6400/60000]
loss: 2.280349  [12800/60000]
loss: 2.274196  [19200/60000]
loss: 2.244789  [25600/60000]
loss: 2.222665  [32000/60000]
loss: 2.226822  [38400/60000]
loss: 2.192845  [44800/60000]
loss: 2.185779  [51200/60000]
loss: 2.162460  [57600/60000]
Test Error: 
 Accuracy: 47.9%, Avg loss: 2.155445 

Epoch 2
-------------------------------
loss: 2.156797  [    0/60000]
loss: 2.154324  [ 6400/60000]
loss: 2.096290  [12800/60000]
loss: 2.117149  [19200/60000]
loss: 2.051962  [25600/60000]
loss: 1.997124  [32000/60000]
loss: 2.032897  [38400/60000]
loss: 1.944755  [44800/60000]
loss: 1.950071  [51200/60000]
loss: 1.889239  [57600/60000]
Test Error: 
 Accuracy: 57.0%, Avg loss: 1.881781 

Epoch 3
-------------------------------
loss: 1.903060  [    0/60000]
loss: 1.887342  [ 6400/60000]
loss: 1.759700  [12800/60000]
loss: 1.811978  [19200/60000]
loss: 1.684467  [25600/60000]
loss: 1.641123  [32000/600

In [9]:
class CNN(nn.Module):
    def __init__(self, in_shape:int, out_shape:int, activation:nn.Module, loss:nn.Module, optimizer:torch.optim.Optimizer):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            nn.MaxPool2d(2, 2),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(64*5*5, 128),
            activation(),
            nn.Linear(128, 64),
            activation(),
            nn.Linear(64, out_shape)
        )

        self.loss = loss()
        self.optimizer = optimizer(self.parameters(), lr=1e-3)

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [10]:
cnn = CNN(28,10, nn.ReLU, nn.CrossEntropyLoss, torch.optim.SGD).to(device)

print(cnn)

CNN(
  (layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Flatten(start_dim=1, end_dim=-1)
    (6): Linear(in_features=1600, out_features=128, bias=True)
    (7): ReLU()
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): ReLU()
    (10): Linear(in_features=64, out_features=10, bias=True)
  )
  (loss): CrossEntropyLoss()
)


In [11]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, cnn)
    test(test_dataloader, cnn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298096  [    0/60000]
loss: 2.300416  [ 6400/60000]
loss: 2.303086  [12800/60000]
loss: 2.307810  [19200/60000]
loss: 2.293981  [25600/60000]
loss: 2.307972  [32000/60000]
loss: 2.296725  [38400/60000]
loss: 2.297654  [44800/60000]
loss: 2.296938  [51200/60000]
loss: 2.305483  [57600/60000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.294675 

Epoch 2
-------------------------------
loss: 2.290601  [    0/60000]
loss: 2.292481  [ 6400/60000]
loss: 2.292264  [12800/60000]
loss: 2.298085  [19200/60000]
loss: 2.285969  [25600/60000]
loss: 2.295231  [32000/60000]
loss: 2.288346  [38400/60000]
loss: 2.287748  [44800/60000]
loss: 2.287618  [51200/60000]
loss: 2.292444  [57600/60000]
Test Error: 
 Accuracy: 19.9%, Avg loss: 2.282740 

Epoch 3
-------------------------------
loss: 2.281223  [    0/60000]
loss: 2.282657  [ 6400/60000]
loss: 2.277858  [12800/60000]
loss: 2.284862  [19200/60000]
loss: 2.274658  [25600/60000]
loss: 2.278093  [32000/600